In [ ]:
%reset -f

In [ ]:
!pip install -q transformers sentencepiece datasets evaluate accelerate transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 76.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import json
import torch
import transformers
from transformers import pipeline, ViTImageProcessor, ViTForImageClassification, ViTModel, AutoTokenizer, T5Tokenizer, MT5Model, AutoFeatureExtractor, MT5ForConditionalGeneration
import pandas as pd
from torchvision.io import read_image
from PIL import Image
from torchvision.transforms import ToTensor
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
from datasets import Dataset, load_from_disk
import evaluate
import gc
from torch.utils.data import DataLoader
from tqdm import tqdm

In [ ]:
gc.collect()
torch.cuda.empty_cache()

We want to use a ViT + mT5 model to work try mulitlingual VQA. ViT is a vision model that has an encoder transformer model like BERT. mT5 is a multilingual seq to seq transformer model (encoder - decoder).

To combine these two models, need to introduce the visual information into the question.



In [ ]:
import os

if os.getcwd() == '/content':
  from google.colab import drive
  drive.mount('/content/drive')
  train_val_dataset = load_from_disk('/content/drive/Shareddrives/CS263_final/models/data/evjvqa_train_PIL_image')
else:
  train_val_dataset = load_from_disk('G:/Shared drives/CS263_final/models/data/evjvqa_train_PIL_image')


Mounted at /content/drive


In [ ]:

en_list = list(range(5702, 7204))
vi_list = list(range(14023, 15524))
ja_list = list(range(22283, 23785))

In [ ]:
val_list = en_list + vi_list + ja_list

In [ ]:
en_list = list(range(4001, 5201))
ja_list = list(range(20582, 21783))
vi_list = list(range(11322, 13522))

In [ ]:
test_list = en_list + vi_list + ja_list

In [ ]:
full_list = list(range(0, 23785))

In [ ]:
train_list = [i for i in full_list if ((i not in test_list) and (i not in val_list))]

In [ ]:
test_dataset = train_val_dataset.select(test_list)
train_dataset = train_val_dataset.select(train_list)
val_dataset = train_val_dataset.select(val_list)

In [ ]:
test_dataset

Dataset({
    features: ['id', 'question', 'answer', 'image'],
    num_rows: 4601
})

In [ ]:
train_dataset

Dataset({
    features: ['id', 'question', 'answer', 'image'],
    num_rows: 14679
})

In [ ]:
val_dataset

Dataset({
    features: ['id', 'question', 'answer', 'image'],
    num_rows: 4505
})

In [ ]:
gc.collect()

44

In [ ]:
txt_checkpoint = "google/mt5-small"
tokenizer = T5Tokenizer.from_pretrained(txt_checkpoint)

img_checkpoint = 'google/vit-base-patch16-224'
image_processor = ViTImageProcessor.from_pretrained(img_checkpoint)

In [ ]:
class MultimodalCollator:
  tokenizer: AutoTokenizer
  preprocessor: AutoFeatureExtractor
  def __init__(self, tokenizer, preprocessor):
    self.tokenizer = tokenizer
    self.preprocessor = preprocessor
  def tokenize_text(self, texts, targets):

      encoded_question = self.tokenizer(
          text=texts,
          max_length = 80,
          padding='max_length',
          return_tensors='pt',
          return_attention_mask=True,
      )
      encoded_label = self.tokenizer(
          text=targets,
          max_length = 80,
          padding='max_length',
          return_tensors='pt',
          return_attention_mask=True,
      )
      return {
          "input_ids": encoded_question['input_ids'],
          "labels": encoded_label['input_ids'],
          "attention_mask": encoded_question['attention_mask'],
          "decoder_attention_mask": encoded_label['attention_mask']
      }

  def preprocess_images(self, images):
      processed_images = self.preprocessor(
          images,
          return_tensors="pt",
      )
      return {
          "pixel_values": processed_images['pixel_values'],
      }

  def __call__(self, examples):
      return {
          **self.tokenize_text(
              examples['question'] if isinstance(examples, dict) else [i['question'] for i in examples],
              examples['answer'] if isinstance(examples, dict) else [i['answer'] for i in examples]
          ),
          **self.preprocess_images(
              examples['image'] if isinstance(examples, dict) else [i['image'] for i in examples]
          )
      }

In [ ]:
collator = MultimodalCollator(tokenizer, image_processor)

In [ ]:
from transformers import MT5ForConditionalGeneration
image_encoder = ViTModel.from_pretrained(img_checkpoint)

Some weights of the model checkpoint at google/vit-base-patch16-224 were not used when initializing ViTModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.weight', 'vit.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from typing import Optional, Tuple, Union
from transformers.modeling_outputs import Seq2SeqLMOutput, BaseModelOutput

class VQA_Model(MT5ForConditionalGeneration):
  """
  The VQAModel should consist of a image encoder and a multilingual language transformer (encoder, decoder, encoder-decoder).

  The visual_text_block is a module that integrates the encodings from the images and text.
  """

  def __init__(self, config, visualEncoder, img_hidden_dim = 768):
    super().__init__(config)
    self.visualEncoder = visualEncoder
    if img_hidden_dim != config.d_model:
      self.needConvert = True
      self.dim_change = nn.Linear(img_hidden_dim, config.d_model)

  def forward(
        self,
        input_ids: Optional[torch.LongTensor] = None,
        pixel_values: torch.FloatTensor = None,
        attention_mask: Optional[torch.FloatTensor] = None,
        decoder_input_ids: Optional[torch.LongTensor] = None,
        decoder_attention_mask: Optional[torch.BoolTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        decoder_head_mask: Optional[torch.FloatTensor] = None,
        cross_attn_head_mask: Optional[torch.Tensor] = None,
        encoder_outputs: Optional[Tuple[Tuple[torch.Tensor]]] = None,
        past_key_values: Optional[Tuple[Tuple[torch.Tensor]]] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        decoder_inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ):


    use_cache = use_cache if use_cache is not None else self.config.use_cache
    return_dict = return_dict if return_dict is not None else self.config.use_return_dict

    # FutureWarning: head_mask was separated into two input args - head_mask, decoder_head_mask
    if head_mask is not None and decoder_head_mask is None:
        if self.config.num_layers == self.config.num_decoder_layers:
            warnings.warn(__HEAD_MASK_WARNING_MSG, FutureWarning)
            decoder_head_mask = head_mask

    # Encode if needed (training, first prediction pass)
    if encoder_outputs is None:
        # Convert encoder inputs in embeddings if needed
        text_encoder_outputs = self.encoder(
            input_ids=input_ids,
            attention_mask=attention_mask,
            inputs_embeds=inputs_embeds,
            head_mask=head_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        image_encoder_outputs = self.visualEncoder(pixel_values = pixel_values)
        if self.needConvert:
          image_state = self.dim_change(image_encoder_outputs['last_hidden_state'])
        else:
          image_state = image_encoder_outputs['last_hidden_state']
        img_mask = torch.ones((image_state.shape[0], image_state.shape[1])).to(image_state.device)
        hidden_states = torch.cat([image_state, text_encoder_outputs[0]], dim = 1)
        attention_mask = torch.cat([img_mask, attention_mask,], dim = 1)

        encoder_outputs = BaseModelOutput(
            last_hidden_state=hidden_states,
            hidden_states=text_encoder_outputs[1] if len(text_encoder_outputs) > 1 else None,
            attentions=text_encoder_outputs[2] if len(text_encoder_outputs) > 2 else None,
        )
    elif return_dict and not isinstance(encoder_outputs, BaseModelOutput):
        encoder_outputs = BaseModelOutput(
            last_hidden_state=encoder_outputs[0],
            hidden_states=encoder_outputs[1] if len(encoder_outputs) > 1 else None,
            attentions=encoder_outputs[2] if len(encoder_outputs) > 2 else None,
        )

    if self.model_parallel:
        torch.cuda.set_device(self.decoder.first_device)

    if labels is not None and decoder_input_ids is None and decoder_inputs_embeds is None:
        # get decoder inputs from shifting lm labels to the right
        decoder_input_ids = self._shift_right(labels)

    # Set device for model parallelism
    if self.model_parallel:
        torch.cuda.set_device(self.decoder.first_device)
        hidden_states = hidden_states.to(self.decoder.first_device)
        if decoder_input_ids is not None:
            decoder_input_ids = decoder_input_ids.to(self.decoder.first_device)
        if attention_mask is not None:
            attention_mask = attention_mask.to(self.decoder.first_device)
        if decoder_attention_mask is not None:
            decoder_attention_mask = decoder_attention_mask.to(self.decoder.first_device)

    hidden_states = encoder_outputs[0]


    # Decode
    decoder_outputs = self.decoder(
        input_ids=decoder_input_ids,
        attention_mask=decoder_attention_mask,
        inputs_embeds=decoder_inputs_embeds,
        past_key_values=past_key_values,
        encoder_hidden_states=hidden_states,
        encoder_attention_mask=attention_mask,
        head_mask=decoder_head_mask,
        cross_attn_head_mask=cross_attn_head_mask,
        use_cache=use_cache,
        output_attentions=output_attentions,
        output_hidden_states=output_hidden_states,
        return_dict=return_dict,
    )

    sequence_output = decoder_outputs[0]

    # Set device for model parallelism
    if self.model_parallel:
        torch.cuda.set_device(self.encoder.first_device)
        self.lm_head = self.lm_head.to(self.encoder.first_device)
        sequence_output = sequence_output.to(self.lm_head.weight.device)

    if self.config.tie_word_embeddings:
        # Rescale output before projecting on vocab
        # See https://github.com/tensorflow/mesh/blob/fa19d69eafc9a482aff0b59ddd96b025c0cb207d/mesh_tensorflow/transformer/transformer.py#L586
        sequence_output = sequence_output * (self.model_dim**-0.5)

    lm_logits = self.lm_head(sequence_output)

    loss = None
    if labels is not None:
        loss_fct = nn.CrossEntropyLoss(ignore_index=0)
        # move labels to correct device to enable PP
        labels = labels.to(lm_logits.device)
        loss = loss_fct(lm_logits.view(-1, lm_logits.size(-1)), labels.view(-1))
        # TODO(thom): Add z_loss https://github.com/tensorflow/mesh/blob/fa19d69eafc9a482aff0b59ddd96b025c0cb207d/mesh_tensorflow/layers.py#L666

    if not return_dict:
        output = (lm_logits,) + decoder_outputs[1:] + encoder_outputs
        return ((loss,) + output) if loss is not None else output

    return Seq2SeqLMOutput(
        loss=loss,
        logits=lm_logits,
        past_key_values=decoder_outputs.past_key_values,
        decoder_hidden_states=decoder_outputs.hidden_states,
        decoder_attentions=decoder_outputs.attentions,
        cross_attentions=decoder_outputs.cross_attentions,
        encoder_last_hidden_state=encoder_outputs.last_hidden_state,
        encoder_hidden_states=encoder_outputs.hidden_states,
        encoder_attentions=encoder_outputs.attentions,
    )


In [ ]:
from transformers import MT5Config
model = VQA_Model.from_pretrained(txt_checkpoint, image_encoder)

Some weights of VQA_Model were not initialized from the model checkpoint at google/mt5-small and are newly initialized: ['visualEncoder.encoder.layer.5.attention.attention.value.bias', 'visualEncoder.encoder.layer.2.attention.attention.query.weight', 'visualEncoder.encoder.layer.0.layernorm_before.bias', 'visualEncoder.encoder.layer.4.intermediate.dense.weight', 'visualEncoder.encoder.layer.8.attention.attention.query.bias', 'visualEncoder.encoder.layer.10.layernorm_before.weight', 'visualEncoder.encoder.layer.8.intermediate.dense.bias', 'visualEncoder.encoder.layer.7.output.dense.bias', 'visualEncoder.encoder.layer.11.layernorm_after.weight', 'visualEncoder.encoder.layer.9.attention.attention.query.bias', 'visualEncoder.encoder.layer.0.attention.attention.value.weight', 'visualEncoder.encoder.layer.6.output.dense.bias', 'visualEncoder.encoder.layer.8.layernorm_after.bias', 'visualEncoder.encoder.layer.3.attention.attention.query.weight', 'visualEncoder.encoder.layer.3.layernorm_after.

In [ ]:
# Freeze the image and text encoders
for param in model.shared.parameters():
  param.requires_grad = False

for param in model.encoder.parameters():
  param.requires_grad = False

for param in model.visualEncoder.parameters():
  param.requires_grad = False


In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, GenerationConfig
!pip install evaluate[template]

if os.getcwd() == '/content':
  save_location = '/content/drive/Shareddrives/CS263_final/models/ViT_mt5_adjusted_dataset/'
else:
  save_location = 'G:/Shared drives/CS263_final/models/ViT_mt5_adjusted_dataset/'

learning_rate = 3e-4

train_args = Seq2SeqTrainingArguments(
    output_dir = save_location,
    overwrite_output_dir = True,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    evaluation_strategy = "epoch",
    learning_rate = learning_rate,
    num_train_epochs = 4,
    logging_strategy = "epoch",
    save_strategy = "epoch",
    seed = 2023,
    fp16 = False,
    bf16 = False,
    load_best_model_at_end = True,
    generation_config = GenerationConfig.from_pretrained(txt_checkpoint)
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-8)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=100,
                                               gamma=0.9)



class CustomSeq2SeqTrainer(Seq2SeqTrainer):
  def get_train_dataloader(self):
    # build train dataloader
    train_loader = DataLoader(self.train_dataset, batch_size=16, shuffle = True, num_workers = 0, collate_fn = collator)
    return train_loader
  def get_eval_dataloader(self, eval_dataset):
    eval_dataset = eval_dataset if eval_dataset is not None else self.eval_dataset
    eval_loader = DataLoader(eval_dataset, batch_size=8, shuffle = False, num_workers = 0, collate_fn = collator)
    return eval_loader
  def get_test_dataloader(self, test_dataset):
    test_dataset = test_dataset if test_dataset is not None else self.test_dataset
    test_loader = DataLoader(test_dataset, batch_size=8, shuffle = False, num_workers = 0, collate_fn = collator)
    return test_loader

f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
  preds, labels = eval_pred
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
  result = f1_metric.compute(predictions = decoded_preds, references=decoded_labels)
  return result

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 82.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3

In [ ]:
trainer = CustomSeq2SeqTrainer(
                  model,
                  args = train_args,
                 # compute_metrics = compute_metrics,
                  data_collator = collator,
                  train_dataset = train_dataset,
                  eval_dataset = val_dataset,
                  optimizers = (optimizer, scheduler)
                 )
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
torch.cuda.empty_cache()

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,3.934700,1.960869
2,2.414800,1.873964
3,2.283100,1.838284
4,2.244800,1.826722


TrainOutput(global_step=3672, training_loss=2.7193325599554057, metrics={'train_runtime': 5077.0898, 'train_samples_per_second': 11.565, 'train_steps_per_second': 0.723, 'total_flos': 7296811570298880.0, 'train_loss': 2.7193325599554057, 'epoch': 4.0})

In [ ]:
save_location

'G:/Shared drives/CS263_final/models/ViT_mt5_adjusted_dataset/'

In [ ]:
trainer.train(resume_from_checkpoint = '/content/drive/Shareddrives/CS263_final/models/ViT_mt5_adjusted_dataset/checkpoint-3672')

Epoch,Training Loss,Validation Loss


TrainOutput(global_step=3672, training_loss=0.0, metrics={'train_runtime': 11.6028, 'train_samples_per_second': 5060.51, 'train_steps_per_second': 316.476, 'total_flos': 7296811570298880.0, 'train_loss': 0.0, 'epoch': 4.0})

In [ ]:
!pip install torchmetrics
from torchmetrics import F1Score
f1 = F1Score(task = 'multiclass', num_classes = 250112, top_k=1, ignore_index=0)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 13.7 MB/s eta 0:00:00


In [ ]:
config = GenerationConfig.from_pretrained(txt_checkpoint, max_length = 80)
greedy_F1_scores = []
shuffled_set = test_dataset.shuffle(seed = 2023).select(list(range(1000)))
for data in shuffled_set:
  inputs = collator(data)
  for k, v in inputs.items():
    inputs[k] = v.to(device)
  labels = inputs.pop('labels' )
  inputs.pop("decoder_attention_mask")
  generation_output = model.generate(**inputs, generation_config = config , return_dict_in_generate = True, output_scores = True, num_beams = 1)
  pred_str = tokenizer.batch_decode(generation_output['sequences'], skip_special_tokens = True)
  padded_str = tokenizer(pred_str +  [data['answer']], padding = 'longest', return_attention_mask = False, return_tensors='pt')
  padded_pred_token = padded_str['input_ids'][0]
  padded_label_token = padded_str['input_ids'][1]
  score = f1(padded_pred_token, padded_label_token)
  greedy_F1_scores.append(score)
  gc.collect()
  torch.cuda.empty_cache()
greedy_f1_score_avg = np.mean(greedy_F1_scores)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (80) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
greedy_f1_score_avg

0.11457736

In [ ]:
beam4_F1_scores = []
for data in shuffled_set:
  inputs = collator(data)
  for k, v in inputs.items():
    inputs[k] = v.to(device)
  labels = inputs.pop('labels' )
  inputs.pop("decoder_attention_mask")
  generation_output = model.generate(**inputs, generation_config = config, return_dict_in_generate = True, output_scores = True, num_beams = 4)
  pred_str = tokenizer.batch_decode(generation_output['sequences'], skip_special_tokens = True)
  padded_str = tokenizer(pred_str +  [data['answer']], padding = 'longest', return_attention_mask = False, return_tensors='pt')
  padded_pred_token = padded_str['input_ids'][0]
  padded_label_token = padded_str['input_ids'][1]
  score = f1(padded_pred_token, padded_label_token)
  beam4_F1_scores.append(score)
  gc.collect()
  torch.cuda.empty_cache()
beam4_f1_score_avg = np.mean(beam4_F1_scores)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (80) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
beam4_f1_score_avg

0.11874576

In [ ]:
trainer.evaluate()

{'eval_loss': 1.8267217874526978,
 'eval_runtime': 209.9235,
 'eval_samples_per_second': 21.46,
 'eval_steps_per_second': 1.343,
 'epoch': 4.0}

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install langdetect
from nltk.translate import bleu
from nltk.translate.bleu_score import SmoothingFunction
from langdetect import detect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 24.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=0dbc1e89b111f7229fceae7a552f5be579898e3ad7a1182fd62569c90a5e5897
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
import string
smoothie = SmoothingFunction().method4
config = GenerationConfig.from_pretrained(txt_checkpoint, max_length = 80)
shuffled_set = test_dataset.shuffle(seed = 2023).select(list(range(1000)))
def bleu_score(reference, candidate):
  # ignore punctuations
  reference = reference.translate(str.maketrans('', '', string.punctuation))
  candidate = candidate.translate(str.maketrans('', '', string.punctuation))
  ref_list, can_list = [],[]
  # convert string to list of words
  try:
      lang = detect(reference)
  except:
      lang = "error"
      print("error for detecting: ", reference)
  # Detect if the language is Japanese, whose words in answers are not split by space.
  # Need to make it a list of characters for bleu score calculation.
  if lang != "en" or lang != "vi":
    # ignore possible spaces in Japanese sentences
    reference = reference.replace(" ", "")
    candidate = candidate.replace(" ", "")
    for ch in reference:
      ref_list.append(ch)
    for ch in candidate:
      can_list.append(ch)
  else:
    ref_list = reference.split()
    can_list = candidate.split()

  score = bleu([ref_list], can_list,smoothing_function=smoothie, weights = [0.25,0.25,0.25,0.25])
  return score

# Evaluate model performance on test dataset
def test_eval(test_data, model):
  f1_torchmetric, bleu_l, pred_token_l, pred_word_l = [],[],[],[]
  f1_en, f1_ja, f1_vi = [],[],[]
  bleu_en, bleu_ja, bleu_vi = [],[],[]
  f1 = F1Score(task="multiclass", num_classes=250, topk = 1, ignore_index = 0)
  for data in test_data:
    inputs = collator(data)
    for k, v in inputs.items():
      inputs[k] = v.to(device)

    labels = inputs.pop('labels' )
    inputs.pop("decoder_attention_mask")
    generation_output = model.generate(**inputs, generation_config = config, return_dict_in_generate = True, output_scores = True, num_beams = 4)
    pred_str = tokenizer.batch_decode(generation_output['sequences'], skip_special_tokens = True)
    padded_str = tokenizer(pred_str +  [data['answer']], padding = 'longest', return_attention_mask = False, return_tensors='pt')
    padded_pred_token = padded_str['input_ids'][0]
    padded_label_token = padded_str['input_ids'][1]

    # F1 score from torch.metric

    s3 = f1(padded_pred_token, padded_label_token)
    f1_torchmetric.append(s3)

    # Compute bleu score
    s2 = bleu_score(data['answer'], pred_str[0])
    bleu_l.append(s2)

    try:
      lang = detect(data['answer'])
    except:
      lang = "error"
      #print("error for detecting: ", answer)
    if lang == "en":
      f1_en.append(s3)
      bleu_en.append(s2)
    elif lang == "vi":
      f1_vi.append(s3)
      bleu_vi.append(s2)
    else:
      f1_ja.append(s3)
      bleu_ja.append(s2)


  return f1_torchmetric, bleu_l, pred_token_l, pred_word_l, f1_en, f1_ja, f1_vi, bleu_en, bleu_ja, bleu_vi

In [ ]:
f1_torchmetric, bleu_l, pred_token_l, pred_word_l, f1_en, f1_ja, f1_vi, bleu_en, bleu_ja, bleu_vi = test_eval(shuffled_set,model)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (80) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
print("average f1 score: ", sum(f1_torchmetric)/len(f1_torchmetric))
print("average f1 score in en: ", sum(f1_en)/len(f1_en))
print("average f1 score in ja: ", sum(f1_ja)/len(f1_ja))
print("average f1 score in vi: ", sum(f1_vi)/len(f1_vi))
print("average bleu score: ", sum(bleu_l)/len(bleu_l))
print("average bleu score in en: ", sum(bleu_en)/len(bleu_en))
print("average bleu score in ja: ", sum(bleu_ja)/len(bleu_ja))
print("average bleu score in vi: ", sum(bleu_vi)/len(bleu_vi))

average f1 score:  tensor(0.1187)
average f1 score in en:  tensor(0.1293)
average f1 score in ja:  tensor(0.1804)
average f1 score in vi:  tensor(0.0720)
average bleu score:  0.19329738470769095
average bleu score in en:  0.21615040545746905
average bleu score in ja:  0.16883627278173355
average bleu score in vi:  0.19905563350710032


In [ ]:
generation_output = model.generate(**data, max_length = 80)

In [ ]:
torch.cuda.empty_cache()
generation_output

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0')

In [ ]:
class EVJVQA_Dataset(torch.utils.data.Dataset):
  """
  Dataset class for the EVJVQA dataset.
  """
  def __init__(self, annotation_file, img_dir, zip_subpath):
    """
    Inputs:
      annotation_file - file path for the annotation json file
      img_dir - file path for the image zip file
      zip_subpath - subfolder in the zip folder
    """
    with open(annotation_file, encoding = 'utf-8') as f:
      json_file = json.load(f)
    self.annotations = pd.DataFrame(json_file['annotations'])
    self.img_reference = pd.DataFrame(json_file['images']).set_index('id')
    self.img_dir = img_dir
    self.transform = ToTensor()
    self.subpath = zip_subpath + '/'

  def __len__(self):
    return len(self.annotations)

  def __getitem__(self, idx):
    annot_id = self.annotations.loc[idx, 'id']
    image_id = self.annotations.loc[idx , 'image_id']
    question = self.annotations.loc[idx , 'question']
    answer = self.annotations.loc[idx , 'answer']
    img_file = self.img_reference.loc[image_id, 'filename']
    with zipfile.ZipFile(self.img_dir, 'r') as zip_ref:
       imgdata = zip_ref.open(self.subpath + img_file)
       img = Image.open(imgdata).convert('RGB')
    img = self.transform(img)
    return {
        'id': annot_id,
        'question': question,
        'answer': answer,
        'image': img,
    }

In [ ]:
annot_path = '/content/drive/Shareddrives/CS263_final/models/data/official_evjvqa_public_test.json'
img_zip_path = '/content/drive/Shareddrives/CS263_final/models/data/public-test-images.zip'
dataset = EVJVQA_Dataset(annot_path, img_zip_path, 'public-test-images')

In [ ]:
loader = DataLoader(dataset, collate_fn = collator)

In [ ]:
config = GenerationConfig.from_pretrained(txt_checkpoint, max_length = 80)
results = {}
for i in dataset:
  id = i['id']
  inputs = collator(i)
  for k, v in inputs.items():
    inputs[k] = v.to(device)
  labels = inputs.pop('labels' )
  inputs.pop("decoder_attention_mask")
  generation_output = model.generate(**inputs, generation_config = config, return_dict_in_generate = True, output_scores = True, num_beams = 4)
  pred_str = tokenizer.batch_decode(generation_output['sequences'], skip_special_tokens = True)
  results[str(id)] = pred_str[0]




/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (80) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/pandas/core/indexes/range.py:391 in get_loc              │
│                                                                                                  │
│    388 │   │   │   if is_integer(key) or (is_float(key) and key.is_integer()):                   │
│    389 │   │   │   │   new_key = int(key)                                                        │
│    390 │   │   │   │   try:                                                                      │
│ ❱  391 │   │   │   │   │   return self._range.index(new_key)                                     │
│    392 │   │   │   │   except ValueError as err:                                                 │
│    393 │   │   │   │   │   raise KeyError(key) from err                                          │
│    394 │   │   │   self._check_indexing_error(key)                                               │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: 5015 is not in range

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 3>:3                                                                              │
│ in __getitem__:24                                                                                │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/pandas/core/indexing.py:1066 in __getitem__              │
│                                                                                                  │
│   1063 │   │   │   key = tuple(list(x) if is_iterator(x) else x for x in key)                    │
│   1064 │   │   │   key = tuple(com.apply_if_callable(x, self.obj) for x in key)                  │
│   1065 │   │   │   if self._is_scalar_access(key):                                               │
│ ❱ 1066 │   │   │   │   return self.obj._get_value(*key, takeable=self._takeable)                 │
│   1067 │   │   │   return self._getitem_tuple(key)                                               │
│   1068 │   │   else:                                                                             │
│   1069 │   │   │   # we by definition only have the 0th axis                                     │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/pandas/core/frame.py:3924 in _get_value                  │
│                                                                                                  │
│    3921 │   │   │   # CategoricalIndex: Trying to use the engine fastpath may give incorrect     │
│    3922 │   │   │   #  results if our categories are integers that dont match our codes          │
│    3923 │   │   │   # IntervalIndex: IntervalTree has no get_loc                                 │
│ ❱  3924 │   │   │   row = self.index.get_loc(index)                                              │
│    3925 │   │   │   return series._values[row]                                                   │
│    3926 │   │                                                                                    │
│    3927 │   │   # For MultiIndex going through engine effectively restricts us to                │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/pandas/core/indexes/range.py:393 in get_loc              │
│                                                                                                  │
│    390 │   │   │   │   try:                                                                      │
│    391 │   │   │   │   │   return self._range.index(new_

In [ ]:
with open('/content/drive/Shareddrives/CS263_final/models/data/public_results.json', 'w') as f:
  json.dump(results, f)

In [ ]:
tokenizer.batch_decode(torch.argmin(outputs['logits'], dim = 2))

['<extra_id_71>痞㲟 <extra_id_98> <extra_id_98> <extra_id_13>\u2b74 <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_71> <extra_id_